In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

In [2]:
print(tf.__version__)

2.18.0


In [3]:
data = pd.read_csv(r"C:\Users\LENOVO\Documents\Presentation_mini\car_customers web.csv")
data.head()

,CarModel,BodyType,Manufacturer,FuelType,SafetyRating,Price,Budget,DesiredFeatures,Age,FamilySize,Purchased
0,Sonet,SUV,Kia,Diesel,4,1050000,1089469,"Infotainment, Rear Camera",35,6,1
1,Seltos,SUV,Kia,Diesel,5,1200000,1393898,"Sunroof, Infotainment",37,4,1
2,XUV700,SUV,Mahindra,Diesel,5,2300000,2737140,"ADAS, Sunroof",37,7,1
3,City,Sedan,Honda,Petrol,5,1200000,1073712,"Sunroof, ABS",37,4,1
4,Kwid,Hatchback,Renault,Petrol,3,450000,405385,ABS,31,2,0


In [4]:
data.isnull().sum()

CarModel           0
BodyType           0
Manufacturer       0
FuelType           0
SafetyRating       0
Price              0
Budget             0
DesiredFeatures    0
Age                0
FamilySize         0
Purchased          0
dtype: int64

In [5]:
x = data.drop(['CarModel','Manufacturer','Price','Purchased'], axis=1)
x.head()

,BodyType,FuelType,SafetyRating,Budget,DesiredFeatures,Age,FamilySize
0,SUV,Diesel,4,1089469,"Infotainment, Rear Camera",35,6
1,SUV,Diesel,5,1393898,"Sunroof, Infotainment",37,4
2,SUV,Diesel,5,2737140,"ADAS, Sunroof",37,7
3,Sedan,Petrol,5,1073712,"Sunroof, ABS",37,4
4,Hatchback,Petrol,3,405385,ABS,31,2


In [6]:
y = data['CarModel']
y.head()

0     Sonet
1    Seltos
2    XUV700
3      City
4      Kwid
Name: CarModel, dtype: object

In [7]:
encode = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_data = encode.fit_transform(x[['BodyType','FuelType','DesiredFeatures']].replace(-1,1))

In [8]:
scalar = StandardScaler()
num_data = scalar.fit_transform(x[['SafetyRating','Budget','Age','FamilySize']])

In [9]:
print("Categorical Data Shape:", cat_data.shape)
print("Numerical Data Shape:", num_data.shape)

Categorical Data Shape: (76, 24)
Numerical Data Shape: (76, 4)


In [10]:
combine_x = np.hstack([cat_data, num_data])

In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
label_encoder = LabelEncoder()

In [13]:
x_train, x_test, y_train, y_test = train_test_split(combine_x, y, test_size=0.2, random_state=42)

In [14]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(60, 28)
(16, 28)
(60,)
(16,)


In [15]:
# Find unique categories in the target column
unique_classes = data["CarModel"].unique()
print(unique_classes)
print("Number of unique classes:", len(unique_classes))
unique_classes = len(unique_classes)

['Sonet' 'Seltos' 'XUV700' 'City' 'Kwid' 'Verna' 'Glanza' 'Ertiga' 'i20'
 'Swift' 'Ignis' 'Wagon R' 'Compass' 'Kiger' 'Venue' 'Magnite' 'Tiago'
 'Innova Crysta' 'Nexon' 'Harrier' 'Alto' 'Celerio' 'Altroz' 'Brezza'
 'Jazz' 'Creta' 'Fortuner' 'Punch' 'Baleno']
Number of unique classes: 29


In [16]:
print(unique_classes)

29


In [17]:
model = keras.Sequential()
model.add(keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(unique_classes, activation='softmax'))

C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [19]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Convert car model names to numeric labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test) 

In [20]:
from tensorflow.keras.utils import to_categorical

# Convert integer labels to one-hot encoding
y_train_onehot = to_categorical(y_train_encoded, num_classes=29)
y_test_onehot = to_categorical(y_test_encoded, num_classes=29)


In [21]:
model.fit(x_train, y_train_onehot, epochs=100, validation_data=(x_test, y_test_onehot))

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 257ms/step - accuracy: 0.0542 - loss: 3.3762 - val_accuracy: 0.0000e+00 - val_loss: 3.3444
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.0972 - loss: 3.3500 - val_accuracy: 0.0000e+00 - val_loss: 3.3242
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.0868 - loss: 3.3192 - val_accuracy: 0.0000e+00 - val_loss: 3.3049
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.0979 - loss: 3.3005 - val_accuracy: 0.0625 - val_loss: 3.2852
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.1201 - loss: 3.2791 - val_accuracy: 0.1250 - val_loss: 3.2672
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.1854 - loss: 3.2509 - val_accuracy: 0.1875 - val_loss: 3.2486
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.2181 - loss: 3.2140 - val_accuracy: 0.1875 - val_loss: 3.2302
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.2826 - loss: 3.1870 - val_accuracy: 0.187

In [22]:
model.save("Car_Suggestion_model.h5")

In [23]:
import joblib
joblib.dump(encode, "encoder.pkl")
joblib.dump(scalar, "scaler.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

['label_encoder.pkl']

In [24]:
new_customer = pd.DataFrame({
    'BodyType': ['SUV'],  
    'FuelType': ['diesel'],  
    'SafetyRating': [4],  
    'Budget': [1500000],  
    'DesiredFeatures': ['Sunroofs'],  
    'Age': [20],  
    'FamilySize': [4]
})

In [25]:
# Load encoders
encoder = joblib.load("encoder.pkl")
scaler = joblib.load("scaler.pkl")
label_encoder = joblib.load("label_encoder.pkl")

In [26]:
from tensorflow.keras.models import load_model

model = load_model("Car_Suggestion_model.h5")

In [27]:
cat_data = encoder.transform(new_customer[['BodyType','FuelType','DesiredFeatures']])
num_data = scaler.transform(new_customer[['SafetyRating','Budget','Age','FamilySize']])

In [28]:
new = np.hstack([cat_data, num_data])

In [29]:
softmax_output = model.predict(new)

predicted_index = softmax_output.argmax()

car_classes = ['Sonet', 'Seltos', 'XUV700', 'City', 'Kwid', 'Verna', 'Glanza', 'Ertiga', 'i20',
 'Swift', 'Ignis', 'Wagon R', 'Compass', 'Kiger', 'Venue', 'Magnite', 'Tiago',
 'Innova Crysta', 'Nexon', 'Harrier', 'Alto', 'Celerio', 'Altroz', 'Brezza',
 'Jazz', 'Creta', 'Fortuner', 'Punch', 'Baleno']

# Get the predicted car name
predicted_car = car_classes[predicted_index]

print(f"Suggested Car Model: {predicted_car}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Suggested Car Model: Harrier
